In [ ]:
# Mount the google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# go to the folder contains subfolder data/
%cd /content/drive/My\ Drive/microstructure/GNN

In [ ]:
# load the data into the dataset
from __future__ import print_function

import numpy as np
import torch
from torch.utils.data import Dataset
from scipy import sparse

class GraphDataSet(Dataset):
    def __init__(self):
        max_node = 300   # max number of grains/nodes for microstructures in the dataset
        features = 5     # number of features of each grain/node

        for i in range(1, 493):
            # load files
            neighbor_file_path = 'data/structure-{}/neighbor.txt'.format(i) 
            feature_file_path = 'data/structure-{}/feature.txt'.format(i)    
            property_file_path = 'data/structure-{}/property.txt'.format(i)
            neighbor = np.loadtxt(neighbor_file_path)
            feature = np.loadtxt(feature_file_path)
            proprty = np.loadtxt(property_file_path)

            # feature data manipulation
            feature = np.delete(feature, 0, axis=1) # remove the first column (Grain ID)
            feature[:, [3]] = (feature[:, [3]] - np.mean(feature[:, [3]])) / np.std(feature[:, [3]]) # normalize grain size
            feature[:, [4]] = (feature[:, [4]] - np.mean(feature[:, [4]])) / np.std(feature[:, [4]]) # normalize number of neighbors

            # normalize the adjaciency matrix
            np.fill_diagonal(neighbor, 1)  # add the identity matrix
            D=np.sum(neighbor,axis=0)    # calculate the diagnoal element of D
            D_inv=np.diag(np.power(D,-0.5)) # construct D
            neighbor=np.matmul(D_inv, np.matmul(neighbor,D_inv)) # symmetric normalization of adjacency matrix 
            
            # match dimension to the max dimension for neighbors
            result = np.zeros((max_node, max_node))
            result[:neighbor.shape[0], :neighbor.shape[1]] = neighbor
            neighbor = result

            # match dimension to the max dimension for features
            result = np.zeros((max_node, features))
            result[:feature.shape[0], :feature.shape[1]] = feature
            feature = result

            # convert the feature matrix and adjacency matrix to sparse matrix
            feature = sparse.csr_matrix(feature)
            neighbor = sparse.csr_matrix(neighbor)

            #delete data points with negative properties
            proprty = proprty[proprty.min(axis=1)>=0,:]

            #get the dimension of proprty
            num_properties, width = np.shape(proprty)

            # independent variable t, the external field
            t = np.delete(proprty, 1, axis=1)

            # label, the magnetostriction
            label = np.delete(proprty, 0, axis=1)

            #change it to the several data points
            if num_properties == 0:
                multiple_neighbor = []
                multiple_feature = []
            elif num_properties == 1:
                multiple_neighbor = [neighbor]
                multiple_feature = [feature]
            elif num_properties == 2:
                multiple_neighbor = [neighbor, neighbor]
                multiple_feature = [feature, feature]
            elif num_properties == 3:
                multiple_neighbor = [neighbor, neighbor, neighbor]
                multiple_feature = [feature, feature, feature]
            elif num_properties == 4:
                multiple_neighbor = [neighbor, neighbor, neighbor, neighbor]
                multiple_feature = [feature, feature, feature,feature]
            elif num_properties == 5:
                multiple_neighbor = [neighbor, neighbor, neighbor, neighbor, neighbor]
                multiple_feature = [feature, feature, feature, feature, feature] 
                
            # concatenating the matrices
            if i == 1:
                adjacency_matrix = multiple_neighbor
                node_attr_matrix = multiple_feature
                t_matrix = t
                label_matrix = label
            else:
                adjacency_matrix = np.concatenate((adjacency_matrix, multiple_neighbor))
                node_attr_matrix = np.concatenate((node_attr_matrix, multiple_feature))
                t_matrix = np.concatenate((t_matrix, t))
                label_matrix = np.concatenate((label_matrix, label))

        # normalize the independent variable t matrix
        t_matrix = t_matrix / 10000

        # normalize the label matrix
        label_mean = np.mean(label_matrix)
        label_std = np.std(label_matrix)
        label_matrix = (label_matrix - label_mean) / label_std

        # save the mean and standard deviation of label
        norm = np.array([label_mean, label_std])
        np.savez_compressed('norm.npz', norm=norm)

        self.adjacency_matrix = np.array(adjacency_matrix)
        self.node_attr_matrix = np.array(node_attr_matrix)
        self.t_matrix = np.array(t_matrix)
        self.label_matrix = np.array(label_matrix)

        print('--------------------')
        print('Training Data:')
        print('adjacency matrix:\t', self.adjacency_matrix.shape)
        print('node attribute matrix:\t', self.node_attr_matrix.shape)
        print('t matrix:\t\t', self.t_matrix.shape)
        print('label name:\t\t', self.label_matrix.shape)
        print('--------------------')

    def __len__(self):
        return len(self.adjacency_matrix)

    def __getitem__(self, idx):
        adjacency_matrix = self.adjacency_matrix[idx].todense()
        node_attr_matrix = self.node_attr_matrix[idx].todense()
        t_matrix = self.t_matrix[idx]
        label_matrix = self.label_matrix[idx]

        adjacency_matrix = torch.from_numpy(adjacency_matrix)
        node_attr_matrix = torch.from_numpy(node_attr_matrix)
        t_matrix = torch.from_numpy(t_matrix)
        label_matrix = torch.from_numpy(label_matrix)
        return adjacency_matrix, node_attr_matrix, t_matrix, label_matrix


In [ ]:
# functions for dividing the data into different folds
import argparse
import numpy as np
from sklearn.model_selection import KFold

# split the indices into different folds
def split_data(num_folds):
    dataset = GraphDataSet()
    num_of_data = dataset.__len__()
    kf = KFold(n_splits=num_folds, shuffle=True)
    indices = []
    for i, (_, index) in enumerate(kf.split(np.arange(num_of_data))):
        np.random.shuffle(index)
        indices.append(index)
    indices = np.array(indices)
    return indices

# save the indices
def extract_graph_data(out_file_path, indices):
    np.savez_compressed(out_file_path, indices = indices)

In [ ]:
# functions for the GNN model
from __future__ import print_function

import argparse
import time
from collections import OrderedDict
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)
import os
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

def tensor_to_variable(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x.float())


def variable_to_numpy(x):
    if torch.cuda.is_available():
        x = x.cpu()
    x = x.data.numpy()
    return x


# the MPL layer
class Message_Passing(nn.Module):
    def forward(self, x, adjacency_matrix):
        neighbor_nodes = torch.bmm(adjacency_matrix, x)
        logging.debug('neighbor message\t', neighbor_nodes.size())
        #x = x + neighbor_nodes
        logging.debug('x shape\t', x.size())
        return x

# the GNN model
class GraphModel(nn.Module):
    def __init__(self, max_node_num, atom_attr_dim, latent_dim):
        super(GraphModel, self).__init__()

        self.max_node_num = max_node_num    # max number of grains/nodes for microstructures in the dataset
        self.atom_attr_dim = atom_attr_dim  # number of features of each grain/node before passing MPLs
        self.latent_dim = latent_dim        # number of features of each grain/node after passing MPls

        # MPLs
        self.graph_modules = nn.Sequential(OrderedDict([
            ('message_passing_0', Message_Passing()),
            ('dense_0', nn.Linear(self.atom_attr_dim, 50)),
            ('activation_0', nn.Sigmoid()),
            ('message_passing_1', Message_Passing()),
            ('dense_1', nn.Linear(50, self.latent_dim)),
            ('activation_1', nn.Sigmoid()),
        ]))
        
        # FLs
        self.fully_connected = nn.Sequential(
            nn.Linear(self.max_node_num * self.latent_dim + 1, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 1)
        )

        return

    def forward(self, node_attr_matrix, adjacency_matrix, t_matrix):
        node_attr_matrix = node_attr_matrix.float()
        adjacency_matrix = adjacency_matrix.float()
        x = node_attr_matrix
        logging.debug('shape\t', x.size())

        for (name, module) in self.graph_modules.named_children():
            if 'message_passing' in name:
                x = module(x, adjacency_matrix=adjacency_matrix)
            else:
                x = module(x)

        # Before flatten, the size should be [Batch size, max_node_num, latent_dim]
        logging.debug('size of x after GNN\t', x.size())
        # After flatten is the graph representation
        x = x.view(x.size()[0], -1)
        logging.debug('size of x after GNN\t', x.size())

        # Concatenate [x, t]
        x = torch.cat((x, t_matrix), 1)

        x = self.fully_connected(x)
        return x


# model training
def train(model, data_loader,epochs,checkpoint_dir,optimizer,criterion,test_dataloader, running_index):
    #model.train()

    print()
    print("*** Training started! ***")
    print()

    filename="Learning_Output_{}.txt".format(running_index)
    output=open(filename, "w")
    print('Epoch Training_time Training_MSE Testing_MSE',file=output)
    output.flush()

    for epoch in range(epochs):
        model.train()
        # save checkpoints
        if epoch % (epochs / 10) == 0 or epoch == epochs-1:
            torch.save(model.state_dict(), '{}/checkpoint_{}.pth'.format(checkpoint_dir, epoch))

        train_start_time = time.time()

        for batch_id, (adjacency_matrix, node_attr_matrix, t_matrix, label_matrix) in enumerate(data_loader):
            adjacency_matrix = tensor_to_variable(adjacency_matrix)
            node_attr_matrix = tensor_to_variable(node_attr_matrix)
            t_matrix = tensor_to_variable(t_matrix)
            label_matrix = tensor_to_variable(label_matrix)

            optimizer.zero_grad()

            y_pred = model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix) # model prediction
            loss = criterion(y_pred, label_matrix) # calculate loss
            loss.backward()    # back propagation
            optimizer.step()   # update weight

        train_end_time = time.time()
        _, training_MSE = test(model, data_loader, 'Training',False,criterion,running_index)
        _, testing_MSE = test(model, test_dataloader, 'Test', False,criterion,running_index)
        print('%d %.3f %e %e' % (epoch, train_end_time-train_start_time, training_MSE, testing_MSE), file=output)
        output.flush()
    
    output.close()

def test(model, data_loader, test_or_tr, printcond,criterion,running_index):
    model.eval()
    if data_loader is None:
        return None, None

    y_label_list, y_pred_list, total_loss = [], [], 0

    for batch_id, (adjacency_matrix, node_attr_matrix, t_matrix, label_matrix) in enumerate(data_loader):
        adjacency_matrix = tensor_to_variable(adjacency_matrix)
        node_attr_matrix = tensor_to_variable(node_attr_matrix)
        t_matrix = tensor_to_variable(t_matrix)
        label_matrix = tensor_to_variable(label_matrix)

        y_pred = model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix)

        y_label_list.extend(variable_to_numpy(label_matrix))
        y_pred_list.extend(variable_to_numpy(y_pred))

    norm = np.load('norm.npz', allow_pickle=True)['norm']
    label_mean, label_std = norm[0], norm[1]

    # get the original value of true value and predicted value
    y_label_list = np.array(y_label_list) * label_std + label_mean
    y_pred_list = np.array(y_pred_list) * label_std + label_mean

    # calculate the MARE and MSE 
    total_MARE = macro_avg_err(y_pred_list, y_label_list)
    total_mse = criterion(torch.from_numpy(y_pred_list), torch.from_numpy(y_label_list)).item()

    length, w = np.shape(y_label_list)
    if printcond:
        filename="{}_Output_{}.txt".format(test_or_tr, running_index)
        output=open(filename, "w")
        print('{} Set Predictions: '.format(test_or_tr), file=output)
        output.flush()
        print('True_Value Predicted_value',file=output)
        output.flush()
        for i in range(0, length):
            print('%f, %f' % (y_label_list[i], y_pred_list[i]),file=output)
            output.flush()

    return total_MARE, total_mse

def get_data(batch_size,idx_path,running_index,folds):
    indices = np.load(idx_path, allow_pickle=True)['indices']
    test_idx = indices[running_index]
    train_idx = indices[[i for i in range(folds) if i != running_index]]
    train_idx = [item for sublist in train_idx for item in sublist]

    dataset = GraphDataSet()
    train_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                   sampler=SubsetRandomSampler(train_idx))
    test_data = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                                  sampler=SubsetRandomSampler(test_idx))

    return train_data, test_data

def macro_avg_err(Y_prime, Y):
    if type(Y_prime) is np.ndarray:
        return np.sum(np.abs(Y - Y_prime)) / np.sum(np.abs(Y))
    return torch.sum(torch.abs(Y-Y_prime)) / torch.sum(torch.abs(Y))

def GNNmodel(max_node_num=300, atom_attr_dim=5, latent_dim=5, epochs=1000, batch_size=32, learning_rate=1e-4, min_learning_rate=1e-5, seed=123, checkpoint_dir='checkpoints/',running_index=0, folds=10, idx_path='indices.npz'):
    
    # make the directory of checkpoint
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

    torch.manual_seed(seed)

    # Define the model
    model = GraphModel(max_node_num=max_node_num, atom_attr_dim=atom_attr_dim, latent_dim=latent_dim)
    if torch.cuda.is_available():
        model.cuda()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate) # optimizer
    criterion = nn.MSELoss()  # loss

    # get the data
    train_dataloader, test_dataloader = get_data(batch_size=batch_size,idx_path=idx_path,running_index=running_index,folds=folds)

    # train the mode
    train(model=model, data_loader=train_dataloader,epochs=epochs,checkpoint_dir=checkpoint_dir,optimizer=optimizer,criterion=criterion,test_dataloader=test_dataloader,running_index=running_index)
    # predictions on the entire training and test datasets
    train_rel, train_mse = test(model=model, data_loader=train_dataloader, test_or_tr='Training',printcond=True,criterion=criterion,running_index=running_index)
    test_rel, test_mse = test(model=model, data_loader=test_dataloader, test_or_tr='Test', printcond=True,criterion=criterion, running_index=running_index)

    print()
    print('--------------------')
    print()
    print("Running index: {}".format(running_index))
    print("Training Relative Error: {:.3f}%".format(100 * train_rel))
    print("Test Relative Error: {:.3f}%".format(100 * test_rel))
    print("Training MSE: {}".format(train_mse))
    print("Test MSE: {}".format(test_mse))

    return model



In [ ]:
# functions for interpretation
import math
def tensor_to_variable_grad(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x.float(),requires_grad=True)

def variable_to_numpy_grad(x):
    if torch.cuda.is_available():
        x = x.cpu()
    x = x.data.numpy()
    return x

# calculate the gradient of points
def gradient_calculation(adjacency_matrix, node_attr_matrix, t_matrix, model):
    adjacency_matrix=tensor_to_variable_grad(adjacency_matrix)
    node_attr_matrix=tensor_to_variable_grad(node_attr_matrix)
    t_matrix=tensor_to_variable_grad(t_matrix)
    
    label=model(adjacency_matrix=adjacency_matrix, node_attr_matrix=node_attr_matrix, t_matrix=t_matrix)
    label.backward()  # back propagation
    grad_node_attr_matrix=variable_to_numpy_grad(node_attr_matrix.grad) # get the gradient of features
    grad_t_matrix=variable_to_numpy_grad(t_matrix.grad)  # get the gradient of external field
    
    return grad_node_attr_matrix, grad_t_matrix
    
def Intergrated_gradient_calculation(adajacency_matrix, node_attr_matrix, t_matrix, model,steps=200):
    nsize=np.asarray(node_attr_matrix.size())
    tsize=np.asarray(t_matrix.size())
    baseline_node_attr_matrix=torch.zeros((1,nsize[0],nsize[1]))
    baseline_t_matrix=torch.zeros((1,tsize[0]))

    #specify baseline: same with input graph except the Euler angles
    alpha = 0.5*math.pi
    beta = 0.5*math.pi
    gamma = 0.5*math.pi
    for i in range(nsize[0]):
        if node_attr_matrix[i][0] != 0:
            baseline_node_attr_matrix[0][i][0]=alpha
        if node_attr_matrix[i][1] != 0:
            baseline_node_attr_matrix[0][i][1]=beta
        if node_attr_matrix[i][2] != 0:
            baseline_node_attr_matrix[0][i][2]=gamma
        baseline_node_attr_matrix[0][i][3]=node_attr_matrix[i][3]
        baseline_node_attr_matrix[0][i][4]=node_attr_matrix[i][4]

    for i in range(tsize[0]):
        baseline_t_matrix[0][i]=t_matrix[i]
        
    adajacency_matrix=torch.reshape(adajacency_matrix,(1,nsize[0],nsize[0]))

    grad_node_attr_matrix=np.zeros((1,nsize[0],nsize[1]))
    grad_t_matrix=np.zeros((1,tsize[0]))
    
    for step in range(steps):
        temp_node_attr_matrix=np.zeros((1,nsize[0],nsize[1]))
        temp_t_matrix=np.zeros((1,tsize[0]))
        temp_node_attr_matrix=torch.from_numpy(temp_node_attr_matrix)
        temp_t_matrix=torch.from_numpy(temp_t_matrix)
        
        for i in range(nsize[0]):
            for j in range(nsize[1]):
                temp_node_attr_matrix[0][i][j]=baseline_node_attr_matrix[0][i][j]+(node_attr_matrix[i][j]-baseline_node_attr_matrix[0][i][j])*step/steps
                
        for i in range(tsize[0]):
            temp_t_matrix[0][i]=baseline_t_matrix[0][i]+(t_matrix[i]-baseline_t_matrix[0][i])*step/steps


        temp_grad_node_attr_matrix, temp_grad_t_matrix=gradient_calculation(adajacency_matrix, temp_node_attr_matrix, temp_t_matrix, model)
        grad_node_attr_matrix=grad_node_attr_matrix+temp_grad_node_attr_matrix
        grad_t_matrix=grad_t_matrix+temp_grad_t_matrix

    node_attr_matrix=node_attr_matrix.numpy()
    baseline_node_attr_matrix=baseline_node_attr_matrix.numpy()
    t_matrix=t_matrix.numpy()
    baseline_t_matrix=baseline_t_matrix.numpy()
        
    for i in range(nsize[0]):
            for j in range(nsize[1]):
                grad_node_attr_matrix[0][i][j]=(node_attr_matrix[i][j]-baseline_node_attr_matrix[0][i][j])*grad_node_attr_matrix[0][i][j]/steps
                
    for i in range(tsize[0]):
        grad_t_matrix[0][i]=(t_matrix[i]-baseline_t_matrix[0][i])*grad_t_matrix[0][i]/steps

    return grad_node_attr_matrix, grad_t_matrix    
        

In [ ]:
# save and output indices
num_folds=10
out_file_path='indices.npz'
print("Output File Path: {}".format(out_file_path))

indices = split_data(num_folds=num_folds)
extract_graph_data(out_file_path=out_file_path, indices = indices)

print("Data successfully split into {} folds!".format(num_folds))

In [ ]:
for i in range(10):
    model=GNNmodel(latent_dim=5,epochs=1000,batch_size=32,learning_rate=1e-4,min_learning_rate=1e-5,running_index=i)

In [ ]:
# Reload the data for getting interpretation results
from __future__ import print_function

import numpy as np
import torch
import os
from torch.utils.data import Dataset
from scipy import sparse

class GraphDataSet_interpretable(Dataset):
    def __init__(self):
        max_node = 300
        features = 5

        for i in range(1, 493):
            # load files
            neighbor_file_path = 'data/structure-{}/neighbor.txt'.format(i)
            feature_file_path = 'data/structure-{}/feature.txt'.format(i)
            property_file_path = 'data/structure-{}/property.txt'.format(i)
            neighbor = np.loadtxt(neighbor_file_path)
            feature = np.loadtxt(feature_file_path)
            proprty = np.loadtxt(property_file_path)

            # feature data manipulation
            feature = np.delete(feature, 0, axis=1)
            feature[:, [3]] = (feature[:, [3]] - np.mean(feature[:, [3]])) / np.std(feature[:, [3]])
            feature[:, [4]] = (feature[:, [4]] - np.mean(feature[:, [4]])) / np.std(feature[:, [4]])

            # normalize the adjaciency matrix
            np.fill_diagonal(neighbor, 1)
            D=np.sum(neighbor,axis=0)
            D_inv=np.diag(np.power(D,-0.5))
            neighbor=np.matmul(D_inv, np.matmul(neighbor,D_inv))
            
            # match dimension to the max dimension for neighbors
            result = np.zeros((max_node, max_node))
            result[:neighbor.shape[0], :neighbor.shape[1]] = neighbor
            neighbor = result

            # match dimension to the max dimension for features
            result = np.zeros((max_node, features))
            result[:feature.shape[0], :feature.shape[1]] = feature
            feature = result

            feature = sparse.csr_matrix(feature)
            neighbor = sparse.csr_matrix(neighbor)

            #delete negative value of proprty
            proprty = proprty[proprty.min(axis=1)>=0,:]

            #get the dimension of proprty
            num_properties, width = np.shape(proprty)

            # independent variable t
            t = np.delete(proprty, 1, axis=1)
            t = t[num_properties-1,:]   # get the field of the last data point associated with this microstructure
            t = t.reshape((1,1))

            # label
            label = np.delete(proprty, 0, axis=1)
            label = label[num_properties-1,:]  # get the target of the last data point associated with this microstructure
            label = label.reshape((1,1))

            # gradient will not change for different field value. So only using one point for interpretation
            multiple_neighbor = [neighbor]
            multiple_feature = [feature]

            if i == 1:
                adjacency_matrix = multiple_neighbor
                node_attr_matrix = multiple_feature
                t_matrix = t
                label_matrix = label
            else:
                adjacency_matrix = np.concatenate((adjacency_matrix, multiple_neighbor))
                node_attr_matrix = np.concatenate((node_attr_matrix, multiple_feature))
                t_matrix = np.concatenate((t_matrix, t))
                label_matrix = np.concatenate((label_matrix, label))

        # normalize the independent variable t matrix
        t_matrix = t_matrix / 10000

        # normalize the label matrix
        label_mean = np.mean(label_matrix)
        label_std = np.std(label_matrix)
        label_matrix = (label_matrix - label_mean) / label_std

        norm = np.array([label_mean, label_std])
        np.savez_compressed('Interpretable_norm.npz', norm=norm)

        self.adjacency_matrix = np.array(adjacency_matrix)
        self.node_attr_matrix = np.array(node_attr_matrix)
        self.t_matrix = np.array(t_matrix)
        self.label_matrix = np.array(label_matrix)

        print('--------------------')
        print('Training Data:')
        print('adjacency matrix:\t', self.adjacency_matrix.shape)
        print('node attribute matrix:\t', self.node_attr_matrix.shape)
        print('t matrix:\t\t', self.t_matrix.shape)
        print('label name:\t\t', self.label_matrix.shape)
        print('--------------------')

    def __len__(self):
        return len(self.adjacency_matrix)

    def __getitem__(self, idx):
        adjacency_matrix = self.adjacency_matrix[idx].todense()
        node_attr_matrix = self.node_attr_matrix[idx].todense()
        t_matrix = self.t_matrix[idx]
        label_matrix = self.label_matrix[idx]

        adjacency_matrix = torch.from_numpy(adjacency_matrix)
        node_attr_matrix = torch.from_numpy(node_attr_matrix)
        t_matrix = torch.from_numpy(t_matrix)
        label_matrix = torch.from_numpy(label_matrix)
        return adjacency_matrix, node_attr_matrix, t_matrix, label_matrix


In [ ]:
dataset=GraphDataSet_interpretable()

In [ ]:
from numpy import savetxt
i=100
Graph=dataset[i]
adajacency_matrix=Graph[0]
node_attr_matrix=Graph[1]
t_matrix=Graph[2]
grad_node_attr_matrix, grad_t_matrix=Intergrated_gradient_calculation(adajacency_matrix, node_attr_matrix, t_matrix, model,steps=200)
feature_gradient=grad_node_attr_matrix[0]
outputnumber=i+1
savetxt("feature_grad_{0}.csv".format(outputnumber), feature_gradient, delimiter=',')